# Functions Library

In [1]:
%reset -f 
import plotly.graph_objects as go
import plotly.express as px
import os
from itertools import cycle
from plotly.validators.scatter.marker import SymbolValidator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.io as pio
import re
result_df = pd.DataFrame()
result_df_ip = pd.DataFrame()
plot_df_caller = pd.DataFrame()

global rep
global df

def open_file_nf1(file_name):
    global p1, p2
    file_name_only = os.path.basename(file_name)
    
    # Use a regular expression to find the P1_X or P1_6Pro or P1_12 pattern
    match = re.search(r'P1_(?:X|6Pro|12)', file_name_only)
    if match:
        rep = match.group(0)  # This will give you "P1_6Pro" or "P1_X" or "P1_12"
    else:
        rep = None
        print(f"Warning: Could not extract device type from '{file_name_only}'")

    df = pd.read_csv(file_name)
    
    # Split the 'Data' column into separate columns
    df[['P_BAT_P1', 'P_RF_P1', 'P_BAT_P2', 'P_RF_P2', 'useful_data', 'useful_state','count_1','count_2']] = df['P_BAT_P1'].str.split(',', expand=True)
    
    # Convert the data in the new columns to numeric type
    df['P_BAT_P1'] = pd.to_numeric(df['P_BAT_P1'])
    df['P_RF_P1'] = pd.to_numeric(df['P_RF_P1'])
    df['P_BAT_P2'] = pd.to_numeric(df['P_BAT_P2'])
    df['P_RF_P2'] = pd.to_numeric(df['P_RF_P2'])
    df['useful_data'] = pd.to_numeric(df['useful_data'])
    df['useful_state'] = pd.to_numeric(df['useful_state'])
    df['count_1'] = pd.to_numeric(df['count_1'])
    df['count_2'] = pd.to_numeric(df['count_2'])

    # Convert 'm_sec_ms' to datetime format
    df['m_sec_ms'] = pd.to_datetime(df['m_sec_ms'], format='%M:%S.%f').dt.time  # Convert to time object
    df['m_sec_ms'] = pd.to_timedelta(df['m_sec_ms'].astype(str))  # Convert to timedelta

    # Initialize variables for handling time resets
    time_offset = pd.Timedelta(0)  # Cumulative offset for resets
    previous_time = df["m_sec_ms"].iloc[0]  # Initialize with the first timestamp
    absolute_times = []  # Store absolute times

    # Calculate absolute time (handling resets)
    for current_time in df["m_sec_ms"]:
        if current_time < previous_time:  # Detect reset
            time_offset += pd.Timedelta(hours=1)  # Add 1 hour (or appropriate offset)
        absolute_times.append(current_time + time_offset)
        previous_time = current_time

    # Add absolute time to DataFrame
    df["m_sec_ms"] = absolute_times
    
    # Extract minute, second, and millisecond components
    df['minute'] = df['m_sec_ms'].dt.components.minutes
    df['second'] = df['m_sec_ms'].dt.components.seconds
    df['millisecond'] = df['m_sec_ms'].dt.components.milliseconds
    df['total_milliseconds'] = (df['minute'] * 60 * 1000) + (df['second'] * 1000) + df['millisecond']

    # Calculate time differences in seconds
    df["dt"] = df["m_sec_ms"].diff().dt.total_seconds().fillna(0)
    
    # Calculate time in seconds from start
    df['time_sec_abs'] = (df['m_sec_ms'] - df['m_sec_ms'].min()).dt.total_seconds()
    df['minutes'], df['seconds'] = divmod(df['time_sec_abs'], 60)
    df['seconds'], df['milliseconds'] = divmod(df['seconds'], 1)
    df['milliseconds'] *= 1000  # Convert to milliseconds
    df['time_formated_abs'] = df['minutes'].astype(str).str.zfill(2) + ':' + \
                 df['seconds'].astype(int).astype(str).str.zfill(2) + '.' + \
                 df['milliseconds'].astype(int).astype(str).str.zfill(3)
    
    # Calculate log duration
    log_duration = (df["m_sec_ms"].max() - df["m_sec_ms"].min()).total_seconds()
    
    # Calculate samples per second
    count_1_avg = df['count_1'].mean()
    count_2_avg = df['count_2'].mean()
    sps_1 = 1024 / count_1_avg
    sps_2 = 1024 / count_2_avg

    # Extract device types from filename
    parts = file_name_only.split('_')
    p1 = parts[3] if len(parts) > 3 else None  # Add a safety check
    p2 = parts[8] if len(parts) > 8 else None  # Add a safety check
    
    # Determine power column mappings based on device types
    if p1 == 'X' and p2 == '6Pro':
        power_columns = {"P_BAT_P1": "E_BAT_Caller", "P_RF_P1": "E_RF_Caller", "P_BAT_P2": "E_BAT_Callee", "P_RF_P2": "E_RF_Callee"}
    elif p1 == '12' and p2 == '6Pro':
        power_columns = {"P_BAT_P1": "E_BAT_Callee", "P_RF_P1": "E_RF_Callee", "P_BAT_P2": "E_BAT_Caller", "P_RF_P2": "E_RF_Caller"}
    elif p1 == '6Pro' and p2 == '12':
        power_columns = {"P_BAT_P1": "E_BAT_Caller", "P_RF_P1": "E_RF_Caller", "P_BAT_P2": "E_BAT_Callee", "P_RF_P2": "E_RF_Callee"}
    elif p1 == '6Pro' and p2 == 'X':
        power_columns = {"P_BAT_P1": "E_BAT_Callee", "P_RF_P1": "E_RF_Callee", "P_BAT_P2": "E_BAT_Caller", "P_RF_P2": "E_RF_Caller"}
    elif p1 == 'X' and p2 == '12':
        power_columns = {"P_BAT_P1": "E_BAT_Caller", "P_RF_P1": "E_RF_Caller", "P_BAT_P2": "E_BAT_Callee", "P_RF_P2": "E_RF_Callee"}
    elif p1 == '12' and p2 == 'X':
        power_columns = {"P_BAT_P1": "E_BAT_Callee", "P_RF_P1": "E_RF_Callee", "P_BAT_P2": "E_BAT_Caller", "P_RF_P2": "E_RF_Caller"}
    else:
        # Default case
        print(f"Warning: Unknown device combination '{p1}' and '{p2}'. Using default mapping.")
        power_columns = {"P_BAT_P1": "E_BAT_Caller", "P_RF_P1": "E_RF_Caller", "P_BAT_P2": "E_BAT_Callee", "P_RF_P2": "E_RF_Callee"}
    
    # Apply continuous energy integration using trapezoidal rule
    for power_col, energy_col in power_columns.items():
        df[energy_col] = np.zeros(len(df))  # Initialize column with zeros

        for i in range(1, len(df)):
            # Compute instantaneous energy using the trapezoidal rule
            df.at[i, energy_col] = np.trapz(
                [df.at[i-1, power_col], df.at[i, power_col]], 
                x=[df.at[i-1, "m_sec_ms"].total_seconds(), df.at[i, "m_sec_ms"].total_seconds()]
            )
    
    # Create a group column by detecting the 0 -> 1 flip
    df['group'] = (df['useful_data'].diff() == 1).cumsum()
    
    # Set group to NaN where useful_data is 0
    df.loc[df['useful_data'] == 0, 'group'] = None
    
    # Map odd groups to "call_est" and even groups to "call"
    df['group'] = df['group'].map(lambda x: 'call_est' if x % 2 == 1 else 'call' if pd.notna(x) else None)
    
    # Compute the duration of each group in seconds and store in variables
    call_est_duration = df[df['group'] == 'call_est']['dt'].sum()
    call_duration = df[df['group'] == 'call']['dt'].sum()
    total_usefull_data_duration = call_duration + call_est_duration

    return df, sps_1, sps_2, total_usefull_data_duration, call_est_duration, call_duration

# def open_file_nf1(file_name):
#     global p1, p2
#     file_name_only = os.path.basename(file_name)
    
#     # Use a regular expression to find the P1_X or P1_6Pro or P1_12 pattern
#     match = re.search(r'P1_(?:X|6Pro|12)', file_name_only)
#     if match:
#         rep = match.group(0)  # This will give you "P1_6Pro" or "P1_X" or "P1_12"
#     else:
#         rep = None
#         print(f"Warning: Could not extract device type from '{file_name_only}'")

#     df = pd.read_csv(file_name)

    
#     # Split the 'Data' column into separate columns
#     df[['P_BAT_P1', 'P_RF_P1', 'P_BAT_P2', 'P_RF_P2', 'useful_data', 'useful_state','count_1','count_2']] = df['P_BAT_P1'].str.split(',', expand=True)
    
#     # Convert the data in the new columns to numeric type
#     df['P_BAT_P1'] = pd.to_numeric(df['P_BAT_P1'])
#     df['P_RF_P1'] = pd.to_numeric(df['P_RF_P1'])
#     df['P_BAT_P2'] = pd.to_numeric(df['P_BAT_P2'])
#     df['P_RF_P2'] = pd.to_numeric(df['P_RF_P2'])
#     df['useful_data'] = pd.to_numeric(df['useful_data'])
#     df['useful_state'] = pd.to_numeric(df['useful_state'])
#     df['count_1'] = pd.to_numeric(df['count_1'])
#     df['count_2'] = pd.to_numeric(df['count_2'])
#     # df['P_BAT_Caller'] = pd.to_numeric(df['P_BAT_P1'])
#     # df['P_RF_Caller'] = pd.to_numeric(df['P_RF_P1'])
#     # df['P_BAT_Callee'] = pd.to_numeric(df['P_BAT_P2'])
#     # df['P_RF_Callee'] = pd.to_numeric(df['P_RF_P2'])


#     # Convert 'm_sec_ms' to datetime
#     df['m_sec_ms'] = pd.to_datetime(df['m_sec_ms'], format='%M:%S.%f')
    
#     # Extract minute, second, and millisecond components
#     df['minute'] = df['m_sec_ms'].dt.minute
#     df['second'] = df['m_sec_ms'].dt.second
#     df['millisecond'] = df['m_sec_ms'].dt.microsecond // 1000  # Convert microseconds to milliseconds
#     df['total_milliseconds'] = (df['minute'] * 60 * 1000) + (df['second'] * 1000) + df['millisecond']
    
#     # Adjust total milliseconds if necessary
#     previous_value = None
#     for index, current_value in df['total_milliseconds'].items():
#         if previous_value is not None and current_value < previous_value:
#             low_index = df['total_milliseconds'].idxmin()
#             correction_value = df.loc[low_index - 1, 'total_milliseconds']
#             df.loc[low_index:, 'total_milliseconds'] += correction_value
#             min_total_milliseconds = df['total_milliseconds'].min()
#             df['adjusted_total_milliseconds'] = df['total_milliseconds'] - min_total_milliseconds
#             break
#         previous_value = current_value
#     else:
#         min_total_milliseconds = df['total_milliseconds'].min()
#         df['adjusted_total_milliseconds'] = df['total_milliseconds'] - min_total_milliseconds

#     # Convert adjusted total milliseconds back to minutes, seconds, and milliseconds
#     df['seconds'], df['milliseconds'] = divmod(df['adjusted_total_milliseconds'], 1000)
#     df['minutes'], df['seconds'] = divmod(df['seconds'], 60)
#     df['time'] = df['minutes'].astype(str).str.zfill(2) + ':' + \
#                  df['seconds'].astype(str).str.zfill(2) + '.' + \
#                  df['milliseconds'].astype(str).str.zfill(3)
#     df['time'] = pd.to_datetime(df['time'], format='%M:%S.%f')
    
#     # Calculate averages
#     count_1_avg = df['count_1'].mean()
#     count_2_avg = df['count_2'].mean()
    
#     # Calculate time differences
#     df["dt"] = df["m_sec_ms"].diff().dt.total_seconds().fillna(0)
    
#     # Add time_sec_abs and time_abs columns
#     df['time_sec_abs'] = (df['m_sec_ms'] - df['m_sec_ms'].min()).dt.total_seconds()
#     df['minutes'], df['seconds'] = divmod(df['time_sec_abs'], 60)
#     df['seconds'], df['milliseconds'] = divmod(df['seconds'], 1)
#     df['milliseconds'] *= 1000  # Convert to milliseconds
#     df['time_formated_abs'] = df['minutes'].astype(str).str.zfill(2) + ':' + \
#                  df['seconds'].astype(int).astype(str).str.zfill(2) + '.' + \
#                  df['milliseconds'].astype(int).astype(str).str.zfill(3)
    
#     # Calculate log duration
#     log_duration = (df["m_sec_ms"].max() - df["m_sec_ms"].min()).total_seconds()
    
#     # Calculate samples per second
#     sps_1 = 1024 / count_1_avg
#     sps_2 = 1024 / count_2_avg

#     file_name_only = os.path.basename(file_name)
#     parts = file_name_only.split('_')
#     p1 = parts[3] if len(parts) > 3 else None  # Add a safety check
#     p2 = parts[8] if len(parts) > 8 else None  # Add a safety check
    
#     # Fix your conditional logic
#     if p1 == 'X' and p2 == '6Pro':
#         power_columns = {"P_BAT_P1": "E_BAT_Caller", "P_RF_P1": "E_RF_Caller", "P_BAT_P2": "E_BAT_Callee", "P_RF_P2": "E_RF_Callee"}
#     elif p1 == '12' and p2 == '6Pro':
#         power_columns = {"P_BAT_P1": "E_BAT_Callee", "P_RF_P1": "E_RF_Callee", "P_BAT_P2": "E_BAT_Caller", "P_RF_P2": "E_RF_Caller"}
#     elif p1 == '6Pro' and p2 == '12':
#         power_columns = {"P_BAT_P1": "E_BAT_Caller", "P_RF_P1": "E_RF_Caller", "P_BAT_P2": "E_BAT_Callee", "P_RF_P2": "E_RF_Callee"}
#     elif p1 == '6Pro' and p2 == 'X':
#         power_columns = {"P_BAT_P1": "E_BAT_Callee", "P_RF_P1": "E_RF_Callee", "P_BAT_P2": "E_BAT_Caller", "P_RF_P2": "E_RF_Caller"}
#     elif p1 == 'X' and p2 == '12':
#         power_columns = {"P_BAT_P1": "E_BAT_Caller", "P_RF_P1": "E_RF_Caller", "P_BAT_P2": "E_BAT_Callee", "P_RF_P2": "E_RF_Callee"}
#     elif p1 == '12' and p2 == 'X':
#         power_columns = {"P_BAT_P1": "E_BAT_Callee", "P_RF_P1": "E_RF_Callee", "P_BAT_P2": "E_BAT_Caller", "P_RF_P2": "E_RF_Caller"}
#     else:
#         # Default case
#         print(f"Warning: Unknown device type '{rep}'. Using default mapping.")
#         power_columns = {"P_BAT_P1": "E_BAT_Caller", "P_RF_P1": "E_RF_Caller", "P_BAT_P2": "E_BAT_Callee", "P_RF_P2": "E_RF_Callee"}
    
#     # FIX: Move this outside the 'else' block so it always executes
#     for power_col, energy_col in power_columns.items():
#         df[energy_col] = np.zeros(len(df))  # Initialize column with zeros
#         for i in range(1, len(df)):
#             # Compute instantaneous energy using the trapezoidal rule
#             df.at[i, energy_col] = np.trapz(
#                 [df.at[i-1, power_col], df.at[i, power_col]], 
#                 x=[df.at[i-1, "m_sec_ms"].timestamp(), df.at[i, "m_sec_ms"].timestamp()]
#             )
    
    
#     # Create a group column by detecting the 0 -> 1 flip
#     df['group'] = (df['useful_data'].diff() == 1).cumsum()
#     # Set group to NaN where useful_data is 0
#     df.loc[df['useful_data'] == 0, 'group'] = None
#     # Map odd groups to "call_est" and even groups to "call"
#     df['group'] = df['group'].map(lambda x: 'call_est' if x % 2 == 1 else 'call' if pd.notna(x) else None)
#     # Compute the duration of each group in seconds and store in a variable
#     call_est_duration = df[df['group'] == 'call_est']['dt'].sum()
#     call_duration = df[df['group'] == 'call']['dt'].sum()
#     total_usefull_data_duration = call_duration + call_est_duration

#     return df, sps_1, sps_2, total_usefull_data_duration, call_est_duration, call_duration  # Return DataFrame + Mean SPS + Duration + Group Durations

def seek_measurement_dataset_analyze_iph_2ch(file_name):
    global result_df_ip, section_df

    # Load the data
    df, sps_caller, sps_callee, tt_u_du,call_est_duration, call_duration  = open_file_nf1(file_name)
    if 'V_BAT' not in df.columns:
        # print(f"Warning: 'V_BAT' column not found in {file_name}. Skipping calculations involving 'V_BAT'.")
        df['V_BAT'] = 0  # Add a default column with zeros or handle as needed

    section_df = df
    file_name = os.path.basename(file_name)  # Extract filename from path
    filename_parts = file_name.split('_')
    rep = filename_parts[0] if len(filename_parts) >= 1 else None
    caller_device = filename_parts[3] if len(filename_parts) >= 4 else None
    techno_1 = filename_parts[4] if len(filename_parts) >= 5 else None
    voice_techno_1 = filename_parts[5] if len(filename_parts) >= 6 else None
    callee_device = filename_parts[8] if len(filename_parts) >= 9 else None
    techno_2 = filename_parts[9] if len(filename_parts) >= 10 else None
    voice_techno_2 = filename_parts[10] if len(filename_parts) >= 11 else None

    with pd.option_context("mode.copy_on_write", True):
        # Energy consumption Joules in Min
        # Call Establishment
        cum_E_RF_Est_Caller_Jm = df[df['group'] == 'call_est']['E_RF_Caller'].sum()/ (call_est_duration/60)
        cum_E_BAT_Est_Caller_Jm = df[df['group'] == 'call_est']['E_BAT_Caller'].sum()/ (call_est_duration/60)
        cum_E_RF_Est_Callee_Jm = df[df['group'] == 'call_est']['E_RF_Callee'].sum()/ (call_est_duration/60)
        cum_E_BAT_Est_Callee_Jm = df[df['group'] == 'call_est']['E_BAT_Callee'].sum()/ (call_est_duration/60)
        # Call 
        cum_E_RF_Call_Caller_Jm = df[df['group'] == 'call']['E_RF_Caller'].sum()/ (call_duration/60)
        cum_E_BAT_Call_Caller_Jm = df[df['group'] == 'call']['E_BAT_Caller'].sum()/ (call_duration/60)
        cum_E_RF_Call_Callee_Jm = df[df['group'] == 'call']['E_RF_Callee'].sum()/ (call_duration/60)
        cum_E_BAT_Call_Callee_Jm = df[df['group'] == 'call']['E_BAT_Callee'].sum()/ (call_duration/60)

        # Total energy consumption Joules in Min - Call and Call Establishment
        cum_E_RF_Full_call_useful_Caller_Jm = cum_E_RF_Est_Caller_Jm + cum_E_RF_Call_Caller_Jm
        cum_E_BAT_Full_call_useful_Caller_Jm = cum_E_BAT_Est_Caller_Jm + cum_E_BAT_Call_Caller_Jm

        cum_E_RF_Full_call_useful_Callee_Jm = cum_E_RF_Est_Callee_Jm + cum_E_RF_Call_Callee_Jm
        cum_E_BAT_Full_call_useful_Callee_Jm = cum_E_BAT_Est_Callee_Jm + cum_E_BAT_Call_Callee_Jm

        # Total energy consumption - Call and Call Establishment
        cum_E_RF_Full_call_useful_Caller = df[df['group'] == 'call_est']['E_RF_Caller'].sum() + df[df['group'] == 'call']['E_RF_Caller'].sum()
        cum_E_BAT_Full_call_useful_Caller = df[df['group'] == 'call_est']['E_BAT_Caller'].sum() + df[df['group'] == 'call']['E_BAT_Caller'].sum()

        cum_E_RF_Full_call_useful_Callee = df[df['group'] == 'call_est']['E_RF_Callee'].sum() + df[df['group'] == 'call']['E_RF_Callee'].sum()
        cum_E_BAT_Full_call_useful_Callee = df[df['group'] == 'call_est']['E_BAT_Callee'].sum() + df[df['group'] == 'call']['E_BAT_Callee'].sum()

    # Calculate E_RF% BAT if 'V_BAT' is available
    if 'V_BAT' in df.columns:
        Caller_e_rf_percent_bat = (cum_E_RF_Full_call_useful_Caller_Jm / cum_E_BAT_Full_call_useful_Caller_Jm) * 100
        Callee_e_rf_percent_bat = (cum_E_RF_Full_call_useful_Callee_Jm / cum_E_BAT_Full_call_useful_Callee_Jm) * 100

    else:
        e_rf_percent_bat = 0  # Default value if 'V_BAT' is missing

    row = {
        'File name': file_name,
        'Exp Repetition': rep,
        'Caller device': caller_device,
        'TX:Caller RAN Tech': techno_1,
        'Caller Voice Tech': voice_techno_1,

        'Callee device': callee_device,
        'RX:Callee RAN Tech': techno_2, 
        'Callee Voice Tech': voice_techno_2,

        ### Energy Joule per minute ###
        ### Caller ###
        # RF
        'Caller Est E_RF Jm': '{:.2f}'.format(cum_E_RF_Est_Caller_Jm), # Call Establishment RF
        'Caller Call E_RF Jm': '{:.2f}'.format(cum_E_RF_Call_Caller_Jm), # Call RF
        # BAT
        'Caller Est E_BAT Jm': '{:.2f}'.format(cum_E_BAT_Est_Caller_Jm), # Call Establishment BAT
        'Caller Call E_BAT Jm': '{:.2f}'.format(cum_E_BAT_Call_Caller_Jm), # Call BAT
        ### Callee ###
        # RF 
        'Callee Est E_RF Jm': '{:.2f}'.format(cum_E_RF_Est_Callee_Jm),
        'Callee Call E_RF Jm': '{:.2f}'.format(cum_E_RF_Call_Callee_Jm), 
        # BAT
        'Callee Est E_BAT Jm': '{:.2f}'.format(cum_E_BAT_Est_Callee_Jm), # Call Establishment BAT
        'Callee Call E_BAT Jm': '{:.2f}'.format(cum_E_BAT_Call_Callee_Jm), # Call BAT


        ### Full Call Usefull Data Joule Per Min ###
        ### Caller ###
        # RF    
        'Full_Use Caller E_RF Jm': '{:.2f}'.format(cum_E_RF_Full_call_useful_Caller_Jm), # Call Establishment RF
        'Full_Use Callee E_RF Jm': '{:.2f}'.format(cum_E_RF_Full_call_useful_Callee_Jm), # Call Establishment RF
        # BAT
        'Full_Use Caller E_BAT Jm': '{:.2f}'.format(cum_E_BAT_Full_call_useful_Caller_Jm), # Call Establishment BAT
        'Full_Use Callee E_BAT Jm': '{:.2f}'.format(cum_E_BAT_Full_call_useful_Callee_Jm), # Call Establishment BAT


        ### Full Call Usefull Data Joule ###
        ### Caller ###
        # RF
        'Full_Use Caller E_RF J': '{:.2f}'.format(cum_E_RF_Full_call_useful_Caller), # Call Establishment RF
        'Full_Use Callee E_RF J': '{:.2f}'.format(cum_E_RF_Full_call_useful_Callee), # Call Establishment RF
        # BAT
        'Full_Use Caller E_BAT J': '{:.2f}'.format(cum_E_BAT_Full_call_useful_Caller), # Call Establishment BAT
        'Full_Use Callee E_BAT J': '{:.2f}'.format(cum_E_BAT_Full_call_useful_Callee), # Call Establishment BAT


        'Caller E_RF% BAT': '{:.2f}%'.format(Caller_e_rf_percent_bat),
        'Callee E_RF% BAT': '{:.2f}%'.format(Callee_e_rf_percent_bat),


        'Duration Call Establishment': '{:.2f}'.format(call_est_duration),
        'Duration Call': '{:.2f}'.format(call_duration),
        'Total duration sec': '{:.2f}'.format(tt_u_du), 
        'SPS caller': '{:.2f}'.format(sps_caller),
        'SPS callee': '{:.2f}'.format(sps_callee),

    }

    result_df_ip = pd.concat([result_df_ip, pd.DataFrame(row, index=[0])], ignore_index=True)
    return tt_u_du, cum_E_RF_Full_call_useful_Caller, cum_E_BAT_Full_call_useful_Caller, section_df, sps_callee

##############################################
##############################################
######## 6pro_3ch###########
##############################################

def open_file_nf_6pro_3ch(file_name):
    """
    Reads a CSV file, processes timestamps (keeping mm:SS.ms format), computes real SPS, 
    and integrates energy using the trapezoidal rule continuously.

    Returns:
        pd.DataFrame: Processed DataFrame with timestamps, fully populated SPS, and cumulative energy.
        float: Mean SPS.
        float: Count-based mean SPS.
        float: Log duration in seconds.
        float: Call estimation duration.
        float: Call duration.
    """
    df = pd.read_csv(file_name)

    # Split the 'V_BAT' column if it's stored as a single column
    df[['V_BAT','I_BAT','P_BAT','V_BB','I_BB','P_BB','V_PA','I_PA','P_PA','useful_data','useful_state','count']] = df['V_BAT'].str.split(',', expand=True)

    # Convert columns to numeric values
    numeric_cols = ['V_BAT', 'I_BAT', 'P_BAT', 'V_BB', 'I_BB', 'P_BB', 'V_PA', 'I_PA', 'P_PA', 'useful_data', 'useful_state', 'count']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

    # Compute Power RF
    df['P_RF'] = df['P_BB'] + df['P_PA']
    # Convert m_sec_ms to timedelta
    df["m_sec_ms"] = pd.to_datetime(df["m_sec_ms"], format='%M:%S.%f').dt.time  # Convert to time object
    df["m_sec_ms"] = pd.to_timedelta(df["m_sec_ms"].astype(str))  # Convert to timedelta

    # Initialize variables
    time_offset = pd.Timedelta(0)  # Cumulative offset for resets
    previous_time = df["m_sec_ms"].iloc[0]  # Initialize with the first timestamp
    absolute_times = []  # Store absolute times

    # Calculate absolute time
    for current_time in df["m_sec_ms"]:
        if current_time < previous_time:  # Detect reset
            time_offset += pd.Timedelta(hours=1)  # Add 1 hour (or appropriate offset)
        absolute_times.append(current_time + time_offset)
        previous_time = current_time

    # Add absolute time to DataFrame
    df["m_sec_ms"] = absolute_times

    # Compute time difference (dt) in seconds
    df["dt"] = df["m_sec_ms"].diff().dt.total_seconds().fillna(0)

    # Extract full seconds for SPS calculation
    df["time"] = df["m_sec_ms"].dt.components.minutes.astype(str).str.zfill(2) + ":" + \
                   df["m_sec_ms"].dt.components.seconds.astype(str).str.zfill(2)  # "MM:SS"

    # Compute SPS (Samples Per Second) and fill all rows
    sps_per_second = df.groupby("time")["count"].apply(lambda x: x.iloc[-1] - x.iloc[0])
    df["SPS"] = df["time"].map(sps_per_second)

    # Compute mean SPS (ignoring NaN values)
    sps_mean = df["SPS"].mean(skipna=True)
    
    # Calculate SPS using count differences - NEW ADDITION
    # First calculate count differences between samples
    df['count_diff'] = df['count'].diff().fillna(0)
    
    # Calculate SPS for each second using count differences
    df['sps_count'] = 0.0  # Initialize column
    
    # Group by each second and calculate count-based SPS
    for time_val, group in df.groupby('time'):
        if len(group) >= 2:
            # Calculate SPS based on count changes within this second
            count_change = group['count'].iloc[-1] - group['count'].iloc[0]
            time_change = (group['m_sec_ms'].iloc[-1] - group['m_sec_ms'].iloc[0]).total_seconds()
            if time_change > 0:
                sps_value = count_change / time_change
                df.loc[df['time'] == time_val, 'sps_count'] = sps_value
    
    # Calculate mean of count-based SPS
    sps_count_mean = df['sps_count'].replace(0, np.nan).mean(skipna=True)

    df['P_BAT_1'] = pd.to_numeric(df['P_BAT'])
    df['P_RF_1'] = pd.to_numeric(df['P_RF'])
    df['P_PA_1'] = pd.to_numeric(df['P_PA'])
    df['P_BB_1'] = pd.to_numeric(df['P_BB'])

    # Continuous Energy Integration using Trapezoidal Rule
    power_columns = {"P_BAT": "E_BAT", "P_RF": "E_RF", "P_PA": "E_PA", "P_BB": "E_BB", "P_RF": "E_RF"}
    for power_col, energy_col in power_columns.items():
        df[energy_col] = np.zeros(len(df))  # Initialize column with zeros

        for i in range(1, len(df)):
            # Compute instantaneous energy using the trapezoidal rule
            df.at[i, energy_col] = np.trapz(
                [df.at[i-1, power_col], df.at[i, power_col]], 
                x=[df.at[i-1, "m_sec_ms"].total_seconds(), df.at[i, "m_sec_ms"].total_seconds()]
            )

    # Add time_sec_abs and time_abs columns
    df['time_sec_abs'] = (df['m_sec_ms'] - df['m_sec_ms'].min()).dt.total_seconds()
    df['minutes'], df['seconds'] = divmod(df['time_sec_abs'], 60)
    df['seconds'], df['milliseconds'] = divmod(df['seconds'], 1)
    df['milliseconds'] *= 1000  # Convert to milliseconds
    df['time_formated_abs'] = df['minutes'].astype(str).str.zfill(2) + ':' + \
                 df['seconds'].astype(int).astype(str).str.zfill(2) + '.' + \
                 df['milliseconds'].astype(int).astype(str).str.zfill(3)
    log_duration = (df["m_sec_ms"].max() - df["m_sec_ms"].min()).total_seconds()

    # Create a group column by detecting the 0 -> 1 flip
    df['group'] = (df['useful_data'].diff() == 1).cumsum()

    # Set group to NaN where useful_data is 0
    df.loc[df['useful_data'] == 0, 'group'] = None

    # Map odd groups to "call_est" and even groups to "call"
    df['group'] = df['group'].map(lambda x: 'call_est' if x % 2 == 1 else 'call' if pd.notna(x) else None)
    
    # Compute the duration of each group in seconds and store in a variable
    call_est_duration = df[df['group'] == 'call_est']['dt'].sum()
    call_duration = df[df['group'] == 'call']['dt'].sum()

    return df, sps_mean, sps_count_mean, log_duration, call_est_duration, call_duration


    


# def open_file_nf_6pro_3ch(file_name):
#     """
#     Reads a CSV file, processes timestamps (keeping mm:SS.ms format), computes real SPS, 
#     and integrates energy using the trapezoidal rule continuously.

#     Returns:
#         pd.DataFrame: Processed DataFrame with timestamps, fully populated SPS, and cumulative energy.
#         float: Mean SPS.
#         float: Log duration in seconds.
#     """
#     df = pd.read_csv(file_name)

#     # Split the 'V_BAT' column if it's stored as a single column
#     df[['V_BAT','I_BAT','P_BAT','V_BB','I_BB','P_BB','V_PA','I_PA','P_PA','useful_data','useful_state','count']] = df['V_BAT'].str.split(',', expand=True)

#     # Convert columns to numeric values
#     numeric_cols = ['V_BAT', 'I_BAT', 'P_BAT', 'V_BB', 'I_BB', 'P_BB', 'V_PA', 'I_PA', 'P_PA', 'useful_data', 'useful_state', 'count']
#     df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

#     # Compute Power RF
#     df['P_RF'] = df['P_BB'] + df['P_PA']
#     # Convert m_sec_ms to timedelta
#     df["m_sec_ms"] = pd.to_datetime(df["m_sec_ms"], format='%M:%S.%f').dt.time  # Convert to time object
#     df["m_sec_ms"] = pd.to_timedelta(df["m_sec_ms"].astype(str))  # Convert to timedelta

#     # Initialize variables
#     time_offset = pd.Timedelta(0)  # Cumulative offset for resets
#     previous_time = df["m_sec_ms"].iloc[0]  # Initialize with the first timestamp
#     absolute_times = []  # Store absolute times

#     # Calculate absolute time
#     for current_time in df["m_sec_ms"]:
#         if current_time < previous_time:  # Detect reset
#             time_offset += pd.Timedelta(hours=1)  # Add 1 hour (or appropriate offset)
#         absolute_times.append(current_time + time_offset)
#         previous_time = current_time

# # Add absolute time to DataFrame
#     df["m_sec_ms"] = absolute_times

# # Compute time difference (dt) in seconds
#     df["dt"] = df["m_sec_ms"].diff().dt.total_seconds().fillna(0)


#     # Extract full seconds for SPS calculation
#     df["time"] = df["m_sec_ms"].dt.components.minutes.astype(str).str.zfill(2) + ":" + \
#                    df["m_sec_ms"].dt.components.seconds.astype(str).str.zfill(2)  # "MM:SS"

#     # Compute SPS (Samples Per Second) and fill all rows
#     sps_per_second = df.groupby("time")["count"].apply(lambda x: x.iloc[-1] - x.iloc[0])
#     df["SPS"] = df["time"].map(sps_per_second)

#     # Compute mean SPS (ignoring NaN values)
#     sps_mean = df["SPS"].mean(skipna=True)
#     df['P_BAT_1'] = pd.to_numeric(df['P_BAT'])
#     df['P_RF_1'] = pd.to_numeric(df['P_RF'])
#     df['P_PA_1'] = pd.to_numeric(df['P_PA'])
#     df['P_BB_1'] = pd.to_numeric(df['P_BB'])


#     # Continuous Energy Integration using Trapezoidal Rule
#     power_columns = {"P_BAT": "E_BAT", "P_RF": "E_RF", "P_PA": "E_PA", "P_BB": "E_BB", "P_RF": "E_RF"}
#     for power_col, energy_col in power_columns.items():
#         df[energy_col] = np.zeros(len(df))  # Initialize column with zeros

#         for i in range(1, len(df)):

#             # Compute instantaneous energy using the trapezoidal rule
#             df.at[i, energy_col] = np.trapz(
#                 [df.at[i-1, power_col], df.at[i, power_col]], 
#                 x=[df.at[i-1, "m_sec_ms"].total_seconds(), df.at[i, "m_sec_ms"].total_seconds()]
#             )

#     # Add time_sec_abs and time_abs columns
#     df['time_sec_abs'] = (df['m_sec_ms'] - df['m_sec_ms'].min()).dt.total_seconds()
#     df['minutes'], df['seconds'] = divmod(df['time_sec_abs'], 60)
#     df['seconds'], df['milliseconds'] = divmod(df['seconds'], 1)
#     df['milliseconds'] *= 1000  # Convert to milliseconds
#     df['time_formated_abs'] = df['minutes'].astype(str).str.zfill(2) + ':' + \
#                  df['seconds'].astype(int).astype(str).str.zfill(2) + '.' + \
#                  df['milliseconds'].astype(int).astype(str).str.zfill(3)
#     log_duration = (df["m_sec_ms"].max() - df["m_sec_ms"].min()).total_seconds()

#     # Create a group column by detecting the 0 -> 1 flip
#     df['group'] = (df['useful_data'].diff() == 1).cumsum()


#     # Set group to NaN where useful_data is 0
#     df.loc[df['useful_data'] == 0, 'group'] = None

#     # Map odd groups to "call_est" and even groups to "call"
#     df['group'] = df['group'].map(lambda x: 'call_est' if x % 2 == 1 else 'call' if pd.notna(x) else None)
#     # Compute the duration of each group in seconds and store in a variable
#     call_est_duration = df[df['group'] == 'call_est']['dt'].sum()
#     call_duration = df[df['group'] == 'call']['dt'].sum()

#     return df, sps_mean, log_duration, call_est_duration,call_duration  # Return DataFrame + Mean SPS + Duration + Group Durations



def seek_measurement_dataset_analyze_6pro_3ch (file_name):
    
    global result_df, section_df   

    global duplicate_rows_result_df
    df,sps,sps_count,duration,du_call_est,du_call = open_file_nf_6pro_3ch(file_name)

    section_df = df
    file_name = os.path.basename(file_name)  # Extract filename from path
    filename_parts = file_name.split('_')
    rep = filename_parts[0] if len(filename_parts) >= 1 else None
    caller_device = filename_parts[2] if len(filename_parts) >= 3 else None
    transmission_1 = filename_parts[3] if len(filename_parts) >= 4 else None
    techno_1 = filename_parts[4] if len(filename_parts) >= 5 else None
    voice_techno_1 = filename_parts[5] if len(filename_parts) >= 6 else None
    callee_device = filename_parts[6] if len(filename_parts) >= 7 else None
    transmission_2 = filename_parts[7] if len(filename_parts) >= 8 else None
    techno_2 = filename_parts[8] if len(filename_parts) >= 9 else None
    voice_techno_2 = filename_parts[9] if len(filename_parts) >= 10 else None

    with pd.option_context("mode.copy_on_write", True):

        # call_du_E_per_min = total_RF/ (total_duration_seconds/60)
        call_E_RF = df[df['group'] == 'call']['E_RF'].sum()
        call_E_BAT = df[df['group'] == 'call']['E_BAT'].sum()
        call_E_PA = df[df['group'] == 'call']['E_PA'].sum()
        call_E_BB = df[df['group'] == 'call']['E_BB'].sum()

        call_est_E_RF = df[df['group'] == 'call_est']['E_RF'].sum()
        call_est_E_BAT = df[df['group'] == 'call_est']['E_BAT'].sum()
        call_est_E_PA = df[df['group'] == 'call_est']['E_PA'].sum()
        call_est_E_BB = df[df['group'] == 'call_est']['E_BB'].sum()

        # Total energy consumption - Call and Call Establishment
        cum_E_RF = call_E_RF + call_est_E_RF
        cum_E_BAT = call_E_BAT + call_est_E_BAT
        cum_E_PA = call_E_PA + call_est_E_PA
        cum_E_BB = call_E_BB + call_est_E_BB

        # Add safety checks before division
        if cum_E_BAT != 0:
            RF_percent_bat = (cum_E_RF / cum_E_BAT) * 100
            PA_percent_bat = (cum_E_PA / cum_E_BAT) * 100
            BB_percent_bat = (cum_E_BB / cum_E_BAT) * 100
        else:
            RF_percent_bat = PA_percent_bat = BB_percent_bat = float('NaN')  # or set to 0 or another default

        if cum_E_RF != 0:
            PA_percent_RF = (cum_E_PA / cum_E_RF) * 100
            BB_percent_RF = (cum_E_BB / cum_E_RF) * 100
        else:
            PA_percent_RF = BB_percent_RF = float('NaN')  # or set to 0 or another default

        duration_est_call = du_call+du_call_est

    row = {
        'File name': file_name,
        'Exp Repetition': rep,
        'Caller device': caller_device,
        'TX:Caller RAN Tech': techno_1,
        'Caller Voice Tech': voice_techno_1,

        'Callee device': callee_device,
        'RX:Callee RAN Tech': techno_2,
        'Callee Voice Tech': voice_techno_2,

        'Call_Est E_BAT Jm': '{:.2f}'.format(call_est_E_BAT/(du_call_est/60)),  # Format power_per_min to three decimal places
        'Call_Est E_RF Jm': '{:.2f}'.format(call_est_E_RF/(du_call_est/60)),  # Format power_per_min to three decimal places
        'Call_Est E_PA Jm': '{:.2f}'.format(call_est_E_PA/(du_call_est/60)),  # Format power_per_min to three decimal places
        'Call_Est E_BB Jm': '{:.2f}'.format(call_est_E_BB/(du_call_est/60)),  # Format power_per_min to three decimal places

        'Call E_BAT Jm': '{:.2f}'.format(call_E_BAT/(du_call/60)),  # Format power_per_min to three decimal places
        'Call E_RF Jm': '{:.2f}'.format(call_E_RF/(du_call/60)),  # Format power_per_min to three decimal places
        'Call E_PA Jm': '{:.2f}'.format(call_E_PA/(du_call/60)),  # Format power_per_min to three decimal places
        'Call E_BB Jm': '{:.2f}'.format(call_E_BB/(du_call/60)),  # Format power_per_min to three decimal places

        'Full_Use E_Bat Jm': '{:.2f}'.format((call_est_E_BAT+call_E_BAT)/(duration_est_call/60)), 
        'Full_Use E_RF Jm': '{:.2f}'.format((call_est_E_RF+call_E_RF)/(duration_est_call/60)),
        'Full_Use E_PA Jm': '{:.2f}'.format((call_est_E_PA+call_E_PA)/(duration_est_call/60)),
        'Full_Use E_BB Jm': '{:.2f}'.format((call_est_E_BB+call_E_BB)/(duration_est_call/60)),

        'Total E_BAT J': '{:.2f}'.format(call_est_E_BAT+call_E_BAT),
        'Total E_RF J': '{:.2f}'.format(call_est_E_RF+call_E_RF),
        'Total E_PA J': '{:.2f}'.format(call_est_E_PA+call_E_PA),
        'Total E_BB J': '{:.2f}'.format(call_est_E_BB+call_E_BB),

        'E_RF % BAT': '{:.2f}%'.format(RF_percent_bat) if not pd.isna(RF_percent_bat) else 'N/A',
        'E_PA % BAT': '{:.2f}%'.format(PA_percent_bat) if not pd.isna(PA_percent_bat) else 'N/A',
        'E_BB % BAT': '{:.2f}%'.format(BB_percent_bat) if not pd.isna(BB_percent_bat) else 'N/A',
        'E_PA % RF': '{:.2f}%'.format(PA_percent_RF) if not pd.isna(PA_percent_RF) else 'N/A',
        'E_BB % RF': '{:.2f}%'.format(BB_percent_RF) if not pd.isna(BB_percent_RF) else 'N/A',

        'Duration Call Establishment': '{:.2f}' .format(du_call_est),
        'Duration Call': '{:.2f}' .format(du_call),
        'Total duration sec': '{:.2f}' .format(duration_est_call),
        'SPS' : '{:.5f}'.format(sps),
        'SPS count' : '{:.5f}'.format(sps_count)
    }
 
    result_df = pd.concat([result_df, pd.DataFrame(row, index=[0])], ignore_index=True)
    duplicate_rows_result_df = result_df[result_df.duplicated()]
    return duration,cum_E_RF,cum_E_BAT, section_df,sps

def apply_exponential_moving_average(data, span):
    smoothed_data = data.ewm(span, adjust=False).mean()
    return smoothed_data

def plot_power_consumption(final_results, device, technology):
    # Define colors for each platform
    # platform_colors = {
    #     'AMAZON': '#146eC4',
    #     'NETFLIX': '#E50924',
    #     'APPLE': 'black',
    #     'DISNEY': '#142874',
    #     'YOUTUBE': '#ff0010'
    # }

    # Filter the final_results DataFrame with two conditions
    filtered_df = final_results[(final_results['Device'] == device) & (final_results['Technology'] == technology)]

    # Check if the filtered dataframe is empty
    if filtered_df.empty:
        print(f"No data available for {device} {technology}")
        return

    # Plotting the bar graph with colors corresponding to platform logos
    fig = px.bar(filtered_df, x='Platform', y='Averg_pm', color='Platform',
                #  color_discrete_map=platform_colors,
                 labels={'Averg_pm': 'Power Consumption per minute (W/Min)', 'Platform': 'Platform'},
                 title=f'Consumption per minute of iPhone {device} across {technology}',
                 height=500, width=700)

    # Set Y-axis limits from 0 to 30
    fig.update_yaxes(range=[0, 30])
    fig.update_layout(showlegend=False)

    # Show the plot
    fig.show()

    # Print platform, number of tests, and total duration for each bar
    print(f"iPhone {device} over {technology}")
    for index, row in filtered_df.iterrows():
        print(f"{row['Platform']}: Num tests: {row['Num_tests']}, Total duration: {row['Total_duration']}")

# Function to preprocess data
def preprocess_data(df):
    df = df.copy()
    df['File name'] = df['File name'].str.replace(r'^\d_5_', '', regex=True)

    non_numeric_cols = [
        'File name', 'Exp Repetition', 'Caller device', 'Callee device',
        'TX:Caller RAN Tech', 'RX:Callee RAN Tech', 'Caller Voice Tech', 'Callee Voice Tech'
    ]

    numeric_cols = df.columns.difference(non_numeric_cols)
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

    grouped = df.groupby('File name')
    mean_numeric = grouped[numeric_cols].mean().reset_index()
    mean_non_numeric = grouped[non_numeric_cols].first().reset_index(drop=True)

    result_df = pd.concat([mean_non_numeric, mean_numeric[numeric_cols]], axis=1)
    return result_df[df.columns]       


# process database

## Android

In [2]:
from tqdm.notebook import tqdm  # Use tqdm.notebook for Jupyter environments

result_df = pd.DataFrame()
unique_filenames = set()
directory_path = "./data/Experiment_Data/Phone Calls Power measurement/db"
#directory_path = '/Users/yobad/Library/CloudStorage/GoogleDrive-youssefmax2.3@gmail.com/My Drive/Phd/Data/Data files/Call power measurement/call_exp_2/'
file_list = [ 
    os.path.join(directory_path, filename) for filename in [
    "1_3_6Pro_TX_E_UMTS_iPhone_RX_3G_UMTS_sps256.csv",
    "1_3_iPhone_TX_4G_VoLTE_6Pro_RX_E_UMTS_sps256.csv",
    "2_3_6Pro_TX_E_UMTS_iPhone_RX_3G_UMTS_sps256.csv",
    "1_3_iPhone_TX_4G_VoWIFI_6Pro_RX_3G_UMTS_sps256.csv",
    "1_3_iPhone_TX_4G_VoWIFI_6Pro_RX_4G_VoLTE_sps256.csv",
    "1_3_iPhone_TX_3G_UMTS_6Pro_RX_E_UMTS_sps256.csv",
    "1_3_iPhone_TX_3G_UMTS_6Pro_RX_4G_VoWIFI_sps256.csv",
    "1_3_iPhone_TX_4G_VoWIFI_6Pro_RX_4G_VoWIFI_sps256.csv",
    "1_3_iPhone_TX_3G_UMTS_6Pro_RX_3G_UMTS_sps256.csv",
    "1_3_iPhone_TX_3G_UMTS_6Pro_RX_4G_VoLTE_sps256.csv",
    "3_3_6Pro_TX_3G_UMTS_iPhone_RX_4G_VoLTE_sps256.csv",
    "2_3_6Pro_TX_3G_UMTS_iPhone_RX_4G_VoLTE_sps256.csv",
    "1_3_6Pro_TX_3G_UMTS_iPhone_RX_4G_VoLTE_sps256.csv",
    "1_3_6Pro_TX_4G_VoWIFI_iPhone_RX_4G_VoLTE_sps256.csv",
    "2_3_6Pro_TX_4G_VoWIFI_iPhone_RX_4G_VoLTE_sps256.csv",
    "3_3_6Pro_TX_4G_VoWIFI_iPhone_RX_4G_VoLTE_sps256.csv",
    "3_3_6Pro_TX_4G_VoLTE_iPhone_RX_4G_VoLTE_sps256.csv",
    "2_3_6Pro_TX_4G_VoLTE_iPhone_RX_4G_VoLTE_sps256.csv",
    "1_3_6Pro_TX_4G_VoLTE_iPhone_RX_4G_VoLTE_sps256.csv",
    "1_3_iPhone_TX_4G_VoLTE_6Pro_RX_3G_UMTS_sps256.csv",
    "1_3_iPhone_TX_4G_VoLTE_6Pro_RX_4G_VoWIFI_sps256.csv",
    "1_3_iPhone_TX_4G_VoLTE_6Pro_RX_4G_VoLTE_sps256.csv",
    "2_3_6Pro_TX_E_UMTS_iPhone_RX_4G_VoLTE_sps256.csv",
    "2_2_iPhone_TX_4G_VoLTE_6Pro_RX_5G_VoLTE_sps256.csv",
    "1_2_iPhone_TX_4G_VoLTE_6Pro_RX_5G_VoLTE_sps256.csv",
    "3_2_iPhone_TX_4G_VoLTE_6Pro_RX_5G_VoLTE_sps256.csv",
    "4_2_iPhone_TX_4G_VoLTE_6Pro_RX_5G_VoLTE_sps256.csv",
    "1_2_6Pro_TX_5G_VoLTE_iPhone_RX_4G_VoLTE_sps256.csv",
    "2_2_6Pro_TX_5G_VoLTE_iPhone_RX_4G_VoLTE_sps256.csv",
    "3_2_6Pro_TX_5G_VoLTE_iPhone_RX_4G_VoLTE_sps256.csv",
    "3_3_6Pro_TX_E_UMTS_iPhone_RX_4G_UMTS_sps256.csv",
    "4_3_6Pro_TX_E_UMTS_iPhone_RX_4G_UMTS_sps256.csv",
    "2_3_iPhone_TX_4G_VoLTE_6Pro_RX_E_UMTS_sps256.csv",
    "3_3_iPhone_TX_4G_VoLTE_6Pro_RX_E_UMTS_sps256.csv",
    "2_3_iPhone_TX_4G_VoLTE_6Pro_RX_3G_UMTS_sps256.csv",
    "old4_3_6Pro_TX_3G_UMTS_iPhone_RX_4G_VoLTE_sps256.csv",
    "4_3_6Pro_TX_3G_UMTS_iPhone_RX_4G_VoLTE_sps256.csv"

    ]
]


files_passed = 0
duplicates_count = 0
problematic_files = []
print(len(file_list))  # This will output 36
for file_name in file_list:
    files_passed += 1
    print(f"{os.path.basename(file_name)}. passed. Count: {files_passed}")
    seek_measurement_dataset_analyze_6pro_3ch(file_name)

37
1_3_6Pro_TX_E_UMTS_iPhone_RX_3G_UMTS_sps256.csv. passed. Count: 1


/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_c

1_3_iPhone_TX_4G_VoLTE_6Pro_RX_E_UMTS_sps256.csv. passed. Count: 2


/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_c

2_3_6Pro_TX_E_UMTS_iPhone_RX_3G_UMTS_sps256.csv. passed. Count: 3


...

/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_c

old4_3_6Pro_TX_3G_UMTS_iPhone_RX_4G_VoLTE_sps256.csv. passed. Count: 36


/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_c

4_3_6Pro_TX_3G_UMTS_iPhone_RX_4G_VoLTE_sps256.csv. passed. Count: 37


/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:489: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_c

## iPhone

In [3]:
directory_path_ip = "./data/Experiment_Data/Phone Calls Power measurement/db"

result_df_ip = pd.DataFrame()

file_list_iPhone = [
    os.path.join(directory_path_ip, filename) for filename in [
    '1_5_P1_12_4G_VoLTE_TO_P2_6Pro_5G_VoLTE_256sps.csv',
    '1_5_P1_12_4G_VoLTE_TO_P2_X_4G_VoLTE_256sps.csv',
    '1_5_P1_12_4G_VoWIFI_TO_P2_X_4G_VoLTE_256sps.csv',
    '1_5_P1_12_5G_VoLTE_TO_P2_X_4G_VoLTE_256sps.csv',
    '1_5_P1_6Pro_5G_VoLTE_TO_P2_X_4G_VoLTE_256sps.csv',
    #'1_5_P1_X_3G_CS_TO_P2_12_4G_VoLTE_10sps.csv',
   # '1_5_P1_X_4G_VoLTE_TO_P2_12_4G_VoLTE_10sps.csv',
   # '1_5_P1_X_4G_VoLTE_TO_P2_12_5G_VoLTE_10sps.csv',
    '1_5_P1_X_4G_VoLTE_TO_P2_12_5G_VoLTE_256sps.csv',
    '1_5_P1_X_4G_VoLTE_TO_P2_6Pro_5G_VoLTE_256sps.csv',
    #'1_5_P1_X_4G_VoWIFI_TO_P2_12_4G_VoWIFI_10sps.csv',
    '2_5_P1_12_4G_VoLTE_TO_P2_6Pro_5G_VoLTE_256sps.csv',
    '2_5_P1_12_4G_VoLTE_TO_P2_X_4G_VoLTE_256sps.csv',
    #'2_5_P1_12_4G_VoWIFI_TO_P2_X_3G_CS_256sps.csv',
    '2_5_P1_12_5G_VoLTE_TO_P2_X_4G_VoLTE_256sps.csv',
    '2_5_P1_6Pro_5G_VoLTE_TO_P2_X_4G_VoLTE_256sps.csv',
    '2_5_P1_X_4G_VoLTE_TO_P2_12_5G_VoLTE_256sps.csv',
    '2_5_P1_X_4G_VoLTE_TO_P2_6Pro_5G_VoLTE_256sps.csv',
    '3_5_P1_12_4G_VoLTE_TO_P2_6Pro_5G_VoLTE_256sps.csv',
    '3_5_P1_12_4G_VoLTE_TO_P2_X_4G_VoLTE_256sps.csv',
    '3_5_P1_12_4G_VoWIFI_TO_P2_X_3G_CS_256sps.csv',
    '3_5_P1_12_5G_VoLTE_TO_P2_X_4G_VoLTE_256sps.csv',
    '3_5_P1_6Pro_5G_VoLTE_TO_P2_X_3G_CS_256sps.csv',
    '3_5_P1_X_3G_CS_TO_P2_6Pro_5G_VoLTE_256sps.csv',
    '3_5_P1_X_4G_VoLTE_TO_P2_12_5G_VoLTE_256sps.csv',
    '3_5_P1_X_4G_VoLTE_TO_P2_6Pro_5G_VoLTE_256sps.csv',
    '4_5_P1_12_4G_VoWIFI_TO_P2_X_3G_CS_256sps.csv',
    '4_5_P1_6Pro_5G_VoLTE_TO_P2_X_3G_CS_256sps.csv',
    '4_5_P1_X_3G_CS_TO_P2_6Pro_5G_VoLTE_256sps.csv',
    '5_5_P1_X_3G_CS_TO_P2_6Pro_5G_VoLTE_256sps.csv',
    '32_5_P1_12_4G_VoWIFI_TO_P2_X_3G_CS_256sps.csv',
    '22_5_P1_12_4G_VoWIFI_TO_P2_X_3G_CS_256sps.csv',
    'o3_5_P1_12_4G_VoWIFI_TO_P2_X_3G_CS_256sps.csv',
    'o2_5_P1_12_4G_VoWIFI_TO_P2_X_3G_CS_256sps.csv',
    '1_3_P1_X_4G_VoLTE_TO_P2_12_4G_VoWIFI_256sps.csv',
    '2_3_P1_X_4G_VoLTE_TO_P2_12_4G_VoWIFI_256sps.csv',
    '3_3_P1_X_4G_VoLTE_TO_P2_12_4G_VoWIFI_256sps.csv'
]

]


files_passed = 0
duplicates_count = 0
problematic_files = []
for file_name in file_list_iPhone:
    files_passed += 1
    print(f"{os.path.basename(file_name)}. passed. Count: {files_passed}")
    seek_measurement_dataset_analyze_iph_2ch(file_name)

1_5_P1_12_4G_VoLTE_TO_P2_6Pro_5G_VoLTE_256sps.csv. passed. Count: 1


/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_c

1_5_P1_12_4G_VoLTE_TO_P2_X_4G_VoLTE_256sps.csv. passed. Count: 2


/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_c

1_5_P1_12_4G_VoWIFI_TO_P2_X_4G_VoLTE_256sps.csv. passed. Count: 3


...

/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_c

2_3_P1_X_4G_VoLTE_TO_P2_12_4G_VoWIFI_256sps.csv. passed. Count: 31


/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_c

3_3_P1_X_4G_VoLTE_TO_P2_12_4G_VoWIFI_256sps.csv. passed. Count: 32


/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_col] = np.trapz(
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/639318165.py:121: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  df.at[i, energy_c

## First Process dataframes 

In [4]:
df = result_df
# Step 1: Clean the 'File name' column by removing prefixes
df['File name'] = df['File name'].str.replace(r'^\d_3_', '', regex=True)
# Step 2: Define non-numeric columns
non_numeric_cols = [
    'File name', 'Exp Repetition', 'Caller device', 'Callee device',
    'TX:Caller RAN Tech', 'RX:Callee RAN Tech', 'Caller Voice Tech', 'Callee Voice Tech'
]
# Step 3: Convert all numeric columns to numeric types
numeric_cols = df.columns.difference(non_numeric_cols)
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Step 4: Group by the cleaned 'File name'
grouped = df.groupby('File name')

# Step 5: Calculate the mean of numeric columns for each group
mean_numeric = grouped[numeric_cols].mean().reset_index()

# Step 6: Add non-numeric columns back to the result
# For non-numeric columns, take the first value in each group (since they are identical)
mean_non_numeric = grouped[non_numeric_cols].first().reset_index(drop=True)

# Step 7: Merge numeric and non-numeric results
mean_result_df = pd.concat([mean_non_numeric, mean_numeric[numeric_cols]], axis=1)

# Step 8: Reorder columns to match the original DataFrame
mean_result_df = mean_result_df[df.columns]

# Define platform colors
Voice_tech_colors = {
    'UMTS': '#146eC4',
    'VoWIFI': '#E51924',
    'VoLTE': 'black'
}

plot_df_caller_6pro = mean_result_df[mean_result_df['Caller device'].str.contains('6Pro')]
plot_df_caller_6pro['6Pro_All_caller'] = plot_df_caller_6pro['TX:Caller RAN Tech'] + '_' + plot_df_caller_6pro['Caller Voice Tech']
plot_df_callee_6pro = mean_result_df[mean_result_df['Caller device'].str.contains('iPhone')]
plot_df_callee_6pro['6Pro_All_callee'] = plot_df_callee_6pro['RX:Callee RAN Tech'] + '_' + plot_df_callee_6pro['Callee Voice Tech']
plot_df_caller_6pro = plot_df_caller_6pro.groupby('6Pro_All_caller', as_index=False).mean(numeric_only=True)
plot_df_callee_6pro = plot_df_callee_6pro.groupby('6Pro_All_callee', as_index=False).mean(numeric_only=True)
plot_df_caller_6pro['6Pro_All_caller'] = plot_df_caller_6pro['6Pro_All_caller'].str.replace('UMTS', 'CS')
plot_df_callee_6pro['6Pro_All_callee'] = plot_df_callee_6pro['6Pro_All_callee'].str.replace('UMTS', 'CS')

###############

# Prepare data for plotting
mean_result_df = preprocess_data(result_df_ip)

# Caller dataframe - remove all callee columns
plot_df_iph_iph_caller = mean_result_df[~mean_result_df['Caller device'].str.contains('6Pro')].copy()
caller_columns_to_exclude = [col for col in mean_result_df.columns if 'Callee' in col]
caller_columns_to_exclude.append('SPS callee')  # Also exclude the SPS callee column
plot_df_iph_iph_caller = plot_df_iph_iph_caller.drop(columns=caller_columns_to_exclude).copy()
plot_df_iph_iph_caller['iP_All_caller'] = plot_df_iph_iph_caller['TX:Caller RAN Tech'] + '_' + plot_df_iph_iph_caller['Caller Voice Tech']

# Callee dataframe - remove all caller columns
plot_df_iph_iph_callee = mean_result_df[~mean_result_df['Callee device'].str.contains('6Pro')].copy()
callee_columns_to_exclude = [col for col in mean_result_df.columns if 'Caller' in col]
callee_columns_to_exclude.append('SPS caller')  # Also exclude the SPS caller column
plot_df_iph_iph_callee = plot_df_iph_iph_callee.drop(columns=callee_columns_to_exclude).copy()
plot_df_iph_iph_callee['iP_All_callee'] = plot_df_iph_iph_callee['RX:Callee RAN Tech'] + '_' + plot_df_iph_iph_callee['Callee Voice Tech']

# Identify numeric columns for caller dataframe
caller_numeric_columns = plot_df_iph_iph_caller.select_dtypes(include=['number']).columns.tolist()

# Calculate averages for caller dataframe by technology type (only for numeric columns)
caller_avg_by_tech = plot_df_iph_iph_caller.groupby('iP_All_caller')[caller_numeric_columns].mean().reset_index()

# Add non-numeric columns that should be preserved (device type, etc.)
# For these columns, we'll take the first occurrence in each group
non_numeric_cols_to_keep_caller = ['Caller device', 'File name', 'TX:Caller RAN Tech', 'Caller Voice Tech']
for col in non_numeric_cols_to_keep_caller:
    if col in plot_df_iph_iph_caller.columns:
        # Use first() to get the first occurrence in each group
        temp_df = plot_df_iph_iph_caller.groupby('iP_All_caller')[col].first().reset_index()
        caller_avg_by_tech = caller_avg_by_tech.merge(temp_df, on='iP_All_caller', how='left')

# Identify numeric columns for callee dataframe
callee_numeric_columns = plot_df_iph_iph_callee.select_dtypes(include=['number']).columns.tolist()

# Calculate averages for callee dataframe by technology type (only for numeric columns)
callee_avg_by_tech = plot_df_iph_iph_callee.groupby('iP_All_callee')[callee_numeric_columns].mean().reset_index()

# Add non-numeric columns that should be preserved
non_numeric_cols_to_keep_callee = ['Callee device', 'File name', 'RX:Callee RAN Tech', 'Callee Voice Tech']
for col in non_numeric_cols_to_keep_callee:
    if col in plot_df_iph_iph_callee.columns:
        # Use first() to get the first occurrence in each group
        temp_df = plot_df_iph_iph_callee.groupby('iP_All_callee')[col].first().reset_index()
        callee_avg_by_tech = callee_avg_by_tech.merge(temp_df, on='iP_All_callee', how='left')





/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/2078688979.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plot_df_caller_6pro['6Pro_All_caller'] = plot_df_caller_6pro['TX:Caller RAN Tech'] + '_' + plot_df_caller_6pro['Caller Voice Tech']
/var/folders/gt/zm5jcrpj6x5fxkp1kx5vrns80000gn/T/ipykernel_37543/2078688979.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plot_df_callee_6pro['6Pro_All_callee'] = plot_df_callee_6pro['RX:Callee RAN Tech'] + '_' + plot_df_callee_6pro['Callee Voi

In [5]:
import pandas as pd
import os

# Load result_df if not already loaded
# result_df = pd.read_csv("path_to_your/result_df_call.csv")

# Create scenario_id column using device, RAN tech, and voice tech
result_df['scenario_id'] = (
    result_df['Caller device'].astype(str).str.strip() + "_" +
    result_df['TX:Caller RAN Tech'].astype(str).str.strip() + "_" +
    result_df['Caller Voice Tech'].astype(str).str.strip()
)

# Clean call-specific energy columns
energy_cols = ['Call E_RF Jm', 'Call E_BAT Jm', 'Call E_BB Jm', 'Call E_PA Jm']
result_df[energy_cols] = result_df[energy_cols].apply(pd.to_numeric, errors='coerce')

# Compute average energy values per scenario
scenario_summary_df = result_df.groupby('scenario_id')[energy_cols].mean().reset_index()

# Rename columns to match frontend format
scenario_summary_df.columns = ['scenario_id', 'E_RF_Jm', 'E_BAT_Jm', 'E_BB_Jm', 'E_PA_Jm']

# Save to CSV in the frontend server folder
os.makedirs("./website/server", exist_ok=True)
scenario_summary_df.to_csv("./website/server/call_scenario_summary_df.csv", index=False)

# Preview first rows
print(scenario_summary_df.head(10))


        scenario_id    E_RF_Jm    E_BAT_Jm    E_BB_Jm    E_PA_Jm
0      6Pro_3G_UMTS  49.560000  139.488000  45.000000   4.562000
1     6Pro_4G_VoLTE  55.396667  160.280000  53.463333   1.933333
2    6Pro_4G_VoWIFI  22.840000  139.416667  21.593333   1.250000
3     6Pro_5G_VoLTE  70.296667  134.676667  60.566667   9.730000
4       6Pro_E_UMTS  54.836000  124.478000  35.128000  19.708000
5    iPhone_3G_UMTS  46.130000  149.365000  39.645000   6.485000
6   iPhone_4G_VoLTE  60.871818  133.698182  45.146364  15.724545
7  iPhone_4G_VoWIFI  43.843333  149.900000  41.306667   2.540000
